In [152]:
import pymysql
import pandas as pd
import re
from bs4 import BeautifulSoup
from hanziconv import HanziConv
from zhon import hanzi
import string
punctuation=hanzi.punctuation + string.punctuation
punctuation = set([i for i in punctuation])
import html
import pymysql
import jieba
from tqdm import tqdm

import nltk
# with open('resources/corpus/stopwords-zh.txt', 'rb') as  file:
#     stopwords = file.read().decode('utf-8').splitlines()
#     stopwords = set(stopwords + nltk.corpus.stopwords.words("english"))
from pyhanlp import HanLP
import json

def extract_text(x):
    soup = BeautifulSoup(x, 'html.parser')
    return soup.text

def clear_text(x):
    x = html.unescape(x)
    x = HanziConv.toSimplified(x)
    x = re.sub(r'\s+', '', x)
    x = x.strip()
    return x

def remove_punctuation(x):
    x = "".join([i for i in x if i not in punctuation])
    return re.sub(r'\s+', ' ', x)

def get_help_content(x):
    x = "".join([i if i not in punctuation else " " for i in x])
    x = re.sub(r'\s+', ' ', x)
    x = x.strip()
    return x

def distinct_count(sentence):
    return len(set([i for i in sentence]))

def tokenize(x):
    x= [i for i in HanLP.segment(x) if i != ' ']
    return x

def join(L):
    return " ".join([i for i in L ])

def from_sql(sql):
    conn = pymysql.connect(host='127.0.0.1',port=3306,user='root',password='eXYhzAWjyvy8grwM',charset='utf8mb4')
    df = pd.read_sql(sql, conn)  
    conn.close()
    return df
def extract_summa(s):
    summa = HanLP.extractSummary(s, 15, r'[ ]')
    return [x for x in summa if 10< len(x) <40]

def extract_s_nature(s):
    return ['{}'.format(term.nature) for term in HanLP.segment(s)]
def clear_summa(summaries):
    return [s for s in summaries if 'c' not in extract_s_nature(s) and 'cc' not in extract_s_nature(s)]

# sql = "SELECT distinct game_id FROM game_source.s_game_comments_taptap_game WHERE source='taptap' limit 1"
# game_id_list = from_sql(sql)
# game_s = pd.DataFrame()
# from datetime import datetime
# t1 = datetime.now().timestamp()
# for game_id in tqdm(list(game_id_list[:100]['game_id'])):
#     sql = """
#     SELECT source, game_id, game_name, content FROM game_source.s_game_comments_taptap_game where game_id = %s and length(content)>300
#     """ % (game_id, )
#     df = from_sql(sql)
    
#     df['content'] = df['content'].apply(clear_text)
# #     df['words'] = df['content'].apply(tokenize)
#     df['content'] = df['content'].apply(get_help_content)
# #     df['clen'] = df['content'].apply(len)
# #     df['cdlen'] = df['content'].apply(distinct_count)
# #     df['crepeat_rate']=df['clen'] / df['cdlen']
# #     df['wlen'] = df['words'].apply(len)
# #     df['wdlen'] = df['words'].apply(distinct_count)
# #     df['wrepeat_rate']=df['wlen'] / df['wdlen']

# #     df_good = df[(df['cdlen'] >= 150) & (df['wrepeat_rate'] < 2)]

# #     if len(df_good) == 0:
# #         continue

#     df['summaries']=df['content'].apply(lambda x: HanLP.extractSummary(x, 1, r'[ ]'))
#     df_new = df[['source', 'game_id', 'game_name', 'summaries']].groupby(['source', 'game_id','game_name']).agg(join).reset_index()
#     #做了两次的提取摘要，
#     df_new['summaries_new'] = df_new['summaries'].apply(lambda x: list(HanLP.extractSummary(x, 15, r'[ ]')))
#     game_s = game_s.append(df_new)
# t2 = datetime.now().timestamp()
# print('本次程序运行分钟：',(t2-t1)/60)

In [165]:
game_id = 70056
sql = """
SELECT source,game_name, game_id, score, up, content FROM game_source.s_game_comments_taptap_game where game_id = %s  and cast(up as signed)>10 
"""% (game_id,)
df = from_sql(sql)

/home/mingchao/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:165: Warning: (1292, "Truncated incorrect INTEGER value: ''")
  result = self._query(query)


In [166]:
df['up'] = df['up'].astype('int16')
df['score'] = df['score'].astype('float16')
df.describe()

,game_id,score,up
count,1065.0,1065.000000,1065.000000
mean,70056.0,4.527344,147.405634
std,0.0,0.893555,266.339110
min,70056.0,1.000000,11.000000
25%,70056.0,4.000000,20.000000
50%,70056.0,5.000000,54.000000
75%,70056.0,5.000000,162.000000
max,70056.0,5.000000,3673.000000


In [167]:
df.sample(5)

,source,game_name,game_id,score,up,content
149,taptap,绝地求生：刺激战场,70056,5.0,241,从末剑过来捧场的\n一搜绝地求生发现腾讯旗下有两款APP我不禁怀疑天美是要和光子大战吗（等等...
853,taptap,绝地求生：刺激战场,70056,4.0,94,谁还记得第一次公测，那种丝滑，那种游戏体验，真滴好，然而一次次更新后，游戏内容丰富了，画质更...
319,taptap,绝地求生：刺激战场,70056,5.0,33,在直播的时候看到了内测，根据很多评论总结出一下意见，望采纳:\n1.玩家反馈最多的就是把小地...
55,taptap,绝地求生：刺激战场,70056,5.0,77,腾讯游戏中的2个工作室我觉得比较好一个是魔方另一个就是光子。（当然现在的魔方也和天美差不多，...
81,taptap,绝地求生：刺激战场,70056,5.0,167,前几天看到了新闻，说腾讯官方宣布代理国服绝地求生，特别激动，终于不用月供加速器了，而且外挂也...


In [168]:
df.shape

(1065, 6)

In [169]:
df['content'] = df['content'].apply(clear_text)
df['format_content'] = df['content'].apply(get_help_content)
df.head(1)

,source,game_name,game_id,score,up,content,format_content
0,taptap,绝地求生：刺激战场,70056,5.0,12,首先一星给还原度一星给体验度三星给工作室（还原度和体验度这么高光子工作室功不可没）-----...,首先一星给还原度一星给体验度三星给工作室 还原度和体验度这么高光子工作室功不可没 接下来是个...


In [170]:
df['summaries']=df['format_content'].apply(lambda x: list(HanLP.extractSummary(x, 1, r'[ ]'))[0])
df.head()

,source,game_name,game_id,score,up,content,format_content,summaries
0,taptap,绝地求生：刺激战场,70056,5.0,12,首先一星给还原度一星给体验度三星给工作室（还原度和体验度这么高光子工作室功不可没）-----...,首先一星给还原度一星给体验度三星给工作室 还原度和体验度这么高光子工作室功不可没 接下来是个...,希望能增加扭头动作和更多动作的结合
1,taptap,绝地求生：刺激战场,70056,4.0,11,现在外挂太猖狂了啊，演都不演的。昨天在圣马丁，我在楼顶听见队友那边有枪声，然后我瞄过去，看见...,现在外挂太猖狂了啊 演都不演的 昨天在圣马丁 我在楼顶听见队友那边有枪声 然后我瞄过去 看见...,看见一个大哥从楼底直接瞬移到我队友面前把我队友杀掉
2,taptap,绝地求生：刺激战场,70056,5.0,98,从s1到现在s3中期，我也是玩了几个赛季了，也一点点见证了刺激战场的进步和完善。记得去年十一...,从s1到现在s3中期 我也是玩了几个赛季了 也一点点见证了刺激战场的进步和完善 记得去年十一...,没那么多时间玩游戏了
3,taptap,绝地求生：刺激战场,70056,4.0,301,首先游乐园在开始的时候不管是在附近还是在场内都会严重卡顿，后来不动了确实是好了。手雷太慢了，...,首先游乐园在开始的时候不管是在附近还是在场内都会严重卡顿 后来不动了确实是好了 手雷太慢了 ...,你们确实应该好好打一局端游了
4,taptap,绝地求生：刺激战场,70056,5.0,24,测试结束第一时间就开始玩，优化还是不错的，我当时联发科mt6750一样能玩，就是有点小卡，现...,测试结束第一时间就开始玩 优化还是不错的 我当时联发科mt6750一样能玩 就是有点小卡 现...,说是您举报的玩家已被扣除信誉积分


In [171]:
# len('太好玩了，打发时间的最佳游戏！力挺')
# content 10<len(content)<40 不需要提取摘要，本身就是摘要
df_content = df[(df['content'].map(len)>10) & (df['content'].map(len)<40)]
for index,x in df_content.iterrows():
    print("score:%s  %s"%(x['score'],x['content']))

score:5.0  光子的就支持一下，天美就算了
score:5.0  不错不错，很喜欢，还原的很好。支持光子。
score:5.0  游戏出来以后我觉得我手指不够用,划不过来?️，同感的赞同
score:5.0  现在还有人想着国外好吗，只有中国才有正版授权手游
score:5.0  说氪金的人真是搞笑，人家逼你冲钱了吗
score:5.0  米6全高玩了一下真的不错~
score:5.0  真的非常期待。平时自己挺羡慕别人的，家里电脑可以玩。自己家电脑卡得要死要死。
score:4.0  天美不错，所以我选择光子
score:5.0  每一个吃鸡的封面都是这个厚颜无耻的3级头？欺负我没捡过？
score:5.0  我想知道这个游戏什么时候出来？?宝宝们谁知道告诉我一声呗
score:5.0  比起天美，我还是喜欢闪电工作室的。
score:5.0  天美老贼丧天良，我与光子共存亡！
score:5.0  啥时候出呀！让我们做好准备买机票坐飞机。
score:5.0  在很多年以后我孙子来对我说爷爷你预约的绝地求生可以下载了我激动的闭上了双眼
score:5.0  这个游戏啥子时候发呦好着急哦
score:4.0  玩了两把，还行，我诺基亚6，430处理器，都能玩流畅，
score:5.0  唉，突然想起了光荣使命hh
score:5.0  脸崩了，就什么都崩了，希望对脸部可以多留点心
score:5.0  期待腾讯吃鸡！快点出来啊等不及了点我让腾讯知道！
score:5.0  磨逼蹭吊，什么时候能出来？磨磨唧唧的。
score:5.0  什么时候才公测啊，都等了一个月了，很多小伙伴心都凉了，百日制作都这么慢吗？
score:4.0  请求更新单独队友禁麦功能，
score:5.0  给我五星支持一下，希望不像旁边天美
score:5.0  光子nb！刚刚在某站看了内测视频觉得完全吊打隔壁啊期待值999+
score:5.0  有谁跟我一样想到今天有得玩，昨天一晚上没睡好？
score:5.0  希望苹果5c也能玩，，，，然后修复一些bug，就完美了
score:5.0  大吉大利，今晚吃鸡！！
score:5.0  刺激是不想要中低端玩家了吗？很想玩啊…快点出适配行不
score:3.0  神TM手雷不能向上扔！手雷投掷限制45°是啥意思？坑死我了！
scor

In [172]:
df_content.shape

(43, 8)

In [173]:
#score 来组合摘要
df_summa = df[(df['summaries'].map(len)>10) & (df['summaries'].map(len)<40)]
df_summa.head()

,source,game_name,game_id,score,up,content,format_content,summaries
0,taptap,绝地求生：刺激战场,70056,5.0,12,首先一星给还原度一星给体验度三星给工作室（还原度和体验度这么高光子工作室功不可没）-----...,首先一星给还原度一星给体验度三星给工作室 还原度和体验度这么高光子工作室功不可没 接下来是个...,希望能增加扭头动作和更多动作的结合
1,taptap,绝地求生：刺激战场,70056,4.0,11,现在外挂太猖狂了啊，演都不演的。昨天在圣马丁，我在楼顶听见队友那边有枪声，然后我瞄过去，看见...,现在外挂太猖狂了啊 演都不演的 昨天在圣马丁 我在楼顶听见队友那边有枪声 然后我瞄过去 看见...,看见一个大哥从楼底直接瞬移到我队友面前把我队友杀掉
3,taptap,绝地求生：刺激战场,70056,4.0,301,首先游乐园在开始的时候不管是在附近还是在场内都会严重卡顿，后来不动了确实是好了。手雷太慢了，...,首先游乐园在开始的时候不管是在附近还是在场内都会严重卡顿 后来不动了确实是好了 手雷太慢了 ...,你们确实应该好好打一局端游了
4,taptap,绝地求生：刺激战场,70056,5.0,24,测试结束第一时间就开始玩，优化还是不错的，我当时联发科mt6750一样能玩，就是有点小卡，现...,测试结束第一时间就开始玩 优化还是不错的 我当时联发科mt6750一样能玩 就是有点小卡 现...,说是您举报的玩家已被扣除信誉积分
10,taptap,绝地求生：刺激战场,70056,4.0,141,tx出了cf内置吃鸡模式，光荣使命，如今又要出吃鸡官方手游；网易则出了终结者和荒野（gp上终...,tx出了cf内置吃鸡模式 光荣使命 如今又要出吃鸡官方手游 网易则出了终结者和荒野 gp上终...,如今又要出吃鸡官方手游


In [174]:
# df_good = df[df['score']>3.0][['source', 'game_id', 'game_name', 'summaries']].groupby(['source', 'game_id','game_name']).agg(join).reset_index()
# #做了两次的提取摘要，
# df_good['summaries'] = df_good['summaries'].apply(extract_summa)
# # df_good['summaries'] = df_good['summaries'].apply(clear_summa)

In [183]:
df_good = df[df['score']>=3.0][['source', 'game_id', 'game_name', 'score', 'summaries']].reset_index()
df_good = df_good[(df_good['summaries'].map(len)>10) & (df_good['summaries'].map(len)<40)]
for index,x in df_good.sample(10).iterrows():
    print(x['score'],':',x['summaries'])

4.0 : butitstillhassomeproblemstofix
5.0 : 关于吃鸡游戏大部分也都是模式相同而已
5.0 : 进行高度细节优化压榨起处理器性能
4.0 : 风衣就贴在腿上面了很丑
5.0 : 希望不要直接去背包里先
5.0 : 我以前这个手机晚上玩刺激战场没有问题
5.0 : 端游已经不是以前那个端游了
4.0 : 不要学王者搞什么段位重置
5.0 : 对于手机来说整体还原度还算可以在下端游玩家
5.0 : 但是好歹比把装备给垃圾队友好


In [181]:
df_good.shape

(628, 5)

In [184]:
df_bad = df[df['score']<3.0][['source', 'game_id', 'game_name', 'score', 'summaries']].reset_index()
df_bad = df_bad[(df_bad['summaries'].map(len)>10) & (df_bad['summaries'].map(len)<40)]
for index,x in df_bad.sample(10).iterrows():
    print(x['score'],':',x['summaries'])

2.0 : 我不指望B站上的大家都说全军好
1.0 : 所以无论这游戏又多氪金
1.0 : 要不是这个游戏还有点还原端游
1.0 : 这个官方看着办吧毕竟不是端游
1.0 : 同学说他天天拿雷炸队友都没被举报
2.0 : 不更新高清极限帧率和稳定流畅度
2.0 : 隔壁已经开放极限帧率了
1.0 : 并且有很多人说这tm是给别的手机做的广告吧你当玩家是傻瓜啊
1.0 : AI刷新在我的视野里了
1.0 : 在野外遇到这情况没车就是等着机器人捶死
